In [1]:
import os,sys,shlex

In [2]:
# Current working direcory
os.chdir("/home/cloudam/work/tinker_scan")

# TinkerPath
TinkerPath = "/home/cloudam/software/tinker"

#Gaussain Input parameters
keyword = 'opt freq apfd/6-311+g(2d,p)'
solvent = 'chloroform'
charge = 0
multiplicity = 1

# Prefix of the Tinker Scan output file (ARC file)
TinkerScanOutput = "IP4"

In [3]:
#Reads force field parameter file to understand atom notation in the output
def ExtractAtomTypes(TinkerPath):
    # open settings.TinkerPath + 'params/mmff.prm
    atomtypes = []
    atomnums = []
    with open(TinkerPath + '/params/mmff.prm', 'r') as f:
        for line in f:
            if line.split(' ')[0] == 'atom':
                data = shlex.split(line, posix=False)
                atomtypes.append(data[3])
                atomnums.append(int(data[-3]))
    return atomtypes, atomnums

In [4]:
def GetAtomSymbol(AtomNum):
    Lookup = ['H', 'He', 'Li', 'Be', 'B', 'C', 'N', 'O', 'F', 'Ne', 'Na', 'Mg', 'Al', \
              'Si', 'P', 'S', 'Cl', 'Ar', 'K', 'Ca', 'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', \
              'Ni', 'Cu', 'Zn', 'Ga', 'Ge', 'As', 'Se', 'Br', 'Kr', 'Rb', 'Sr', 'Y', 'Zr', \
              'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'In', 'Sn', 'Sb', 'Te', 'I', \
              'Xe', 'Cs', 'Ba', 'La', 'Ce', 'Pr', 'Nd', 'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', \
              'Ho', 'Er', 'Tm', 'Yb', 'Lu', 'Hf', 'Ta', 'W', 'Re', 'Os', 'Ir', 'Pt', 'Au', \
              'Hg', 'Tl', 'Pb', 'Bi', 'Po', 'At', 'Rn']

    if AtomNum > 0 and AtomNum < len(Lookup):
        return Lookup[AtomNum-1]
    else:
        print("No such element with atomic number " + str(AtomNum))
        return 0

In [5]:
def ReadArc(f, atypes, anums):
    conffile = open(f + '.arc', 'r')
    confdata = conffile.readlines()
    conffile.close()
    #output data: conformers - list of x,y,z lists, atoms - list of atoms
    conformers = []
    atoms = []
    atypes = [x[:3] for x in atypes]

    for line in confdata:
        data = [_f for _f in line.split('  ') if _f]
        if len(data) < 3:
            conformers.append([])
        else:
            if len(conformers) == 1:
                anum = anums[atypes.index(data[1][:3])]
                atoms.append(GetAtomSymbol(anum))
            conformers[-1].append([x for x in data[2:5]])

    return atoms, conformers


In [6]:
# Get energies of conformers from tinker output file
def GetEnergiesCharge(TinkerOutput):

    infile = open(TinkerOutput + '.tout', 'r')

    inp = infile.readlines()
    if len(inp) < 56:
        print("Tinker output " + TinkerOutput + " is corrupted, aborting.")
        quit()

    #Get the conformer energies from the file
    energies = []
    for line in inp[13:]:
        data = line[:-1].split('  ')
        data = [_f for _f in data if _f]
        if len(data) >= 3:
            if 'Map' in data[0] and 'Minimum' in data[1]:
                energies.append(float(data[-1]))
                #print data

    infile.close()
    return energies

In [7]:
# extract atomtype from MMFF94 force filed
atomtypes, atomnums = ExtractAtomTypes(TinkerPath)

In [8]:
# Read Tinker Scan ARC
atoms,conformers = ReadArc(TinkerScanOutput,atomtypes,atomnums)

In [9]:
# variables:
# TinkerScanOutput:   the prefix of the Tinker output file. For exanple, IP4 is the prefix name of IP4.arc
# ConfId: is used to defined the conformer file
# keyword: the keywork for Gaussian calculation
# solvent: the PCM solvent
# atoms: atom list returned from ReadArc
# conformer: the conformer extracted from Tinker Arc 

def WriteGaussFile(TinkerScanOutput,ConfId,keyword,solvent,atoms,conformer):
    natom = 0
    ChkFile = '%chk='+TinkerScanOutput+'_'+str(ConfId)+'.chk\n'
    OptRoute = '#p '+keyword+' scrf=(solvent='+solvent+')\n'
    Title = 'Structure: '+TinkerScanOutput+'_'+str(ConfId)+'. Calculation: '+keyword+' \n'
    GaussInput = TinkerScanOutput+'_'+str(ConfId)+'.com'
    atoms = atoms
    conformer = conformer
    f = open(GaussInput,'w')
    f.write(ChkFile)
    f.write(OptRoute)
    f.write("\n")
    f.write(Title)
    f.write("\n")
    f.write(str(charge)+' '+str(multiplicity)+'\n')
    for atom in conformer:
        f.write(atoms[natom] + ' ' + atom[0] + ' ' + atom[1] + ' ' +atom[2]+ '\n')
        natom = natom+1

    f.write('\n')
    f.close()

In [10]:
# extract atomtype from MMFF94 force filed
atomtypes, atomnums = ExtractAtomTypes(TinkerPath)

# Read Tinker Scan ARC
atoms,conformers = ReadArc(TinkerScanOutput,atomtypes,atomnums)


In [11]:
# Gernerate Gaussian input file
for cid in range(len(conformers)):
    conformer = conformers[cid]
    ConfId = cid + 1
    WriteGaussFile(TinkerScanOutput,ConfId,keyword,solvent,atoms,conformer)

In [12]:
# extract conformer energy
energy = GetEnergiesCharge("IP4")
print(energy)

[31.3355, 29.0416, 31.3763]
